In [1]:
import pandas as pd 
import numpy as np 
from transformers import pipeline
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
from peft import get_peft_config, PeftModel, PeftConfig, get_peft_model, LoraConfig, TaskType, prepare_model_for_kbit_training
import evaluate

/Users/dkim1998/Documents/virtual_environment/virtual_environment/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/Users/dkim1998/Documents/virtual_environment/virtual_environment/lib/python3.11/site-packages/transformers/utils/generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
/Users/dkim1998/Documents/virtual_environment/virtual_environment/lib/python3.11/site-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(


In [ ]:
###Game plan: Lets create two test: one using clean PII data and the other using dirty PII data and see if we can train the model
##Also remember to create a training dataset and validation dataset as well

#our virtual environment is this: source /Users/dkim1998/Documents/virtual_environment/virtual_environment/bin/activate

In [ ]:
###Using PEFT and LoRA for fine tuning
#1. https://huggingface.co/docs/peft/task_guides/token-classification-lora
#2. https://magazine.sebastianraschka.com/p/practical-tips-for-finetuning-llms
#3. https://github.com/huggingface/peft

###Using Evaluate to test out machine learning
#1. https://huggingface.co/docs/evaluate/index

###Unstructured data to structured data using LLM
#1. https://positivethinking.tech/insights/llm-mini-series-from-unstructured-data-to-structured-data-with-open-source-llms/


In [3]:
from datasets import load_dataset
from transformers import (
    AutoModelForTokenClassification,
    AutoTokenizer,
    DataCollatorForTokenClassification,
    TrainingArguments,
    Trainer,
)
from peft import get_peft_config, PeftModel, PeftConfig, get_peft_model, LoraConfig, TaskType
import evaluate
import torch
import numpy as np

#model_checkpoint = "roberta-large"
#lr = 1e-3
#batch_size = 16
#num_epochs = 10

In [4]:
##Uses mistral 7B model

tokenizer = AutoTokenizer.from_pretrained("/Users/dkim1998/Documents/Mistral_7b_v2") #Note it seems using 13B use so much memory
model = AutoModelForCausalLM.from_pretrained("/Users/dkim1998/Documents/Mistral_7b_v2")

# Encode some input text
inputs = tokenizer("who has the strongest military in the world in 2023", return_tensors="pt")

# Generate a response
outputs = model.generate(**inputs, max_length=50) #uses up to 30 gigabyte of ram just to run it
generated_text = tokenizer.decode(outputs[0])
print(generated_text)

Loading checkpoint shards: 100%|██████████| 3/3 [00:03<00:00,  1.18s/it]
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


<s> who has the strongest military in the world in 2023?

The United States military remains the strongest in the world in 2023, according to the Global Firepower Index, which ranks military power based on factors


In [7]:
##Uses llama 7B model (use this)

tokenizer = AutoTokenizer.from_pretrained("/Users/dkim1998/Documents/llama7b_Jan_2024_hf_only") #Note it seems using 13B use so much memory
model = AutoModelForCausalLM.from_pretrained("/Users/dkim1998/Documents/llama7b_Jan_2024_hf_only")

# Encode some input text
inputs = tokenizer("who has the strongest military in the world in 2023", return_tensors="pt")

# Generate a response
outputs = model.generate(**inputs, max_length=50) #uses up to 30 gigabyte of ram just to run it
generated_text = tokenizer.decode(outputs[0])
print(generated_text)

Loading checkpoint shards: 100%|██████████| 2/2 [00:04<00:00,  2.15s/it]


<s> who has the strongest military in the world in 2023
The World’s Strongest Military
The United States is the world’s most powerful military, with the largest and most advanced military equipment. The US military


In [5]:
##Lets get llama 7b to find PII data and organize them without finetuning them
file_path = "/Users/dkim1998/Downloads/LLM_PII_Project_Code/fake_pii_data_unlabeled_training.txt"

with open(file_path, 'r') as file:
    text_data = file.read()


In [6]:
#prompt = f"Identify and label the PII in the following text:\n{text_data}"
#prompt = f"Please label the dates of birth and phone numbers in the following text:\n{text_data}"
#prompt = f"""
#I will provide a text, and I need you to identify and label all dates of birth and phone numbers. 
#For each date of birth, prepend it with [DOB], and for each phone number, prepend it with [Phone]. 
#Here's an example for clarification:

#Original: John was born on 05/04/1978 and his phone number is 555-1234.
#Labeled: John was born on [DOB] 05/04/1978 and his phone number is [Phone] 555-1234.

#Now, label the dates of birth and phone numbers in the following text:
#{text_data}
#"""

##Prompt 3: good results 01/07/2024
#prompt = f"""
#In the provided text, there are various pieces of information including names, addresses, and contact details. 
#Your task is to identify and label all phone numbers and dates of birth. 
#Please prepend phone numbers with [Phone] and dates of birth with [DOB]. 
#For example, if a date of birth is '1918-04-02', label it as '[DOB] 1918-04-02'. 
#If a phone number is '747-738-2595', label it as '[Phone] 747-738-2595'.

#Now, please process the following text and label the dates of birth and phone numbers accordingly:

#<Courtney Rodriguez, 4773 Schneider Spring
#East Tim, WY 59293, fconley@example.net, +1-246-477-1167, 707-16-5747, TV 98095, VLJE00438703, VISA 13 digit
#Zachary Wood
#4367458967634 02/29
#CVC: 767
#, YYSD59883155680533, Sawyer-Webb, Control and instrumentation engineer, lucasellis, ^%d9DTQhN#, https://diaz.biz/, 1918-04-02, Talk pretty leg., Almost short best., 27.8301585, -179.231569, Mean he available field. Situation structure threat subject act physical., 978-1-211-42653-7
#Joseph Wood, 12515 Wilson Terrace Suite 185
#Lake Karen, IL 81081, ylyons@example.com, 747.738.2595x38185, 742-52-7953, 71-79040, ERCF57892299, Diners Club / Carte Blanche
#Stephanie Perkins
#30049209050831 01/31
#CVC: 655 > 
#"""

##Prompt 4
prompt = f"""
The following text contains mixed information including names, addresses, emails, dates of birth, and phone numbers. 
Your task is to find and label only the dates of birth and phone numbers. 
Remember, a date of birth typically appears in formats like 'YYYY-MM-DD' or 'MM/DD/YYYY', and phone numbers can appear in formats like 'XXX-XXX-XXXX' or '+XX-XXX-XXX-XXXX'. 

Label each date of birth with [DOB] before it, and each phone number with [Phone]. 
For example:
- '1980-05-23' should be labeled as '[DOB] 1980-05-23'.
- '+1-246-477-1167' should be labeled as '[Phone] +1-246-477-1167'.

Now, carefully examine the following text and apply the labels accurately:

<Courtney Rodriguez, 4773 Schneider Spring
East Tim, WY 59293, fconley@example.net, +1-246-477-1167, 707-16-5747, TV 98095, VLJE00438703, VISA 13 digit
Zachary Wood
4367458967634 02/29
CVC: 767
, YYSD59883155680533, Sawyer-Webb, Control and instrumentation engineer, lucasellis, ^%d9DTQhN#, https://diaz.biz/, 1918-04-02, Talk pretty leg., Almost short best., 27.8301585, -179.231569, Mean he available field. Situation structure threat subject act physical., 978-1-211-42653-7
Joseph Wood, 12515 Wilson Terrace Suite 185
Lake Karen, IL 81081, ylyons@example.com, 747.738.2595x38185, 742-52-7953, 71-79040, ERCF57892299, Diners Club / Carte Blanche
Stephanie Perkins
30049209050831 01/31
CVC: 655> 

"""
# Tokenize and generate a response (rest of the code remains the same)



In [7]:
inputs = tokenizer(prompt, return_tensors="pt", truncation=True, max_length=1024)
outputs = model.generate(**inputs, max_length=1024) # Adjust max_length if needed
generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)

print(generated_text)

##Mistral results gave this:
#[Phone] 747.738.2595x38185
#[Phone] 742-52-7953
#[Phone] 71-79040
#[Phone] +1-246-477-1167
#[Phone] 27.8301585
#[Phone] 978-1-211-42653-7
#[DOB] 1918-04-02
#[DOB] 02/29
#[DOB] 01/31
#takes 2 minutes 30 seconds to run this

##Mistral results gave this part 2:
#[Phone] 747-738-2595
#[Phone] 742-52-7953
#[Phone] 71-79040
#[Phone] 747.738.2595x38185
#[Phone] 978-1-211-42653-7
#[Phone] 1-246-477-1167
#[Phone] 27.8301585
#[Phone] -179.231569
#[DOB] 1918-04-02
#[DOB] 02/29
#[DOB] 01/31
#[Email] fconley@example.net
#[Email] ylyons@example.com
#[Email] lucasellis@example.com
#[Address] 4773 Schneider Spring, East Tim, WY 59293
#[Address] 12515 Wilson Terrace Suite 185, Lake Karen, IL 81081
#[Name] Courtney Rodriguez
#[Name] Zachary Wood
#[Name] Joseph Wood
#[Name] Stephanie Perkins
#[SSN] 4367458967634
#[SSN] 30049209050831
#[VISA] VISA 13 digit
#[VISA] ERCF57892299
#[VISA] Diners Club / Carte Blanche
#[Other] TV 98095
#[Other] VLJE00438703
#[Other] VISA 13 digit
#[Other] CVC: 767
#[Other] CVC: 655
#[Other] YYSD59883155680533

##Mistral results gave this part 3 using prompt 4:
#[DOB] 1918-04-02
#[Phone] +1-246-477-1167
#[Phone] 747.738.2595x38185
#[Phone] 742-52-7953
#[Phone] 71-79040
#[Phone] 978-1-211-42653-7
#[DOB] 02/29/1972
#[CVC] 767
#[CVC] 655

##llama 7b results (no tuning):
#Did not show any of the labels at all #5 min

##llama 7b results 2 (no tuning):
#Did not show any of the labels at all

##llama 7b results 3 with prompt 4 (no tuning):
#No results again


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.



The following text contains mixed information including names, addresses, emails, dates of birth, and phone numbers. 
Your task is to find and label only the dates of birth and phone numbers. 
Remember, a date of birth typically appears in formats like 'YYYY-MM-DD' or 'MM/DD/YYYY', and phone numbers can appear in formats like 'XXX-XXX-XXXX' or '+XX-XXX-XXX-XXXX'. 

Label each date of birth with [DOB] before it, and each phone number with [Phone]. 
For example:
- '1980-05-23' should be labeled as '[DOB] 1980-05-23'.
- '+1-246-477-1167' should be labeled as '[Phone] +1-246-477-1167'.

Now, carefully examine the following text and apply the labels accurately:

<Courtney Rodriguez, 4773 Schneider Spring
East Tim, WY 59293, fconley@example.net, +1-246-477-1167, 707-16-5747, TV 98095, VLJE00438703, VISA 13 digit
Zachary Wood
4367458967634 02/29
CVC: 767
, YYSD59883155680533, Sawyer-Webb, Control and instrumentation engineer, lucasellis, ^%d9DTQhN#, https://diaz.biz/, 1918-04-02, Talk pretty

In [ ]:
###Setting up training data
#For this one we will be using 2000 very dirty labeled text data
training_data = ("/Users/dkim1998/Downloads/fake_pii_data_very_dirty_labeled_testing.txt")


In [4]:
###preparing model for qlora
model = prepare_model_for_kbit_training(model)

In [6]:
###Setting up lora
config = LoraConfig(
    r=16,
    lora_alpha=8,
    target_modules=["q_proj","k_proj","v_proj","o_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, config)

In [10]:
#Loading and Preparing the Dataset
from transformers import AutoTokenizer

def generate_prompt(context):
    return f"<Human>: {context}\n<AI>:"

def generate_and_tokenize_prompt(context):
    full_prompt = generate_prompt(context)
    tokenized_full_prompt = tokenizer(full_prompt, padding=True, truncation=True, return_tensors="pt")
    return tokenized_full_prompt

# Read the text file and process each line
tokenized_prompts = []
with open("/Users/dkim1998/Downloads/fake_pii_data_very_dirty_labeled_testing.txt", "r") as file:
    for line in file:
        # Assuming each line is a separate context
        context = line.strip()
        tokenized_prompt = generate_and_tokenize_prompt(context)
        tokenized_prompts.append(tokenized_prompt)

ValueError: Asking to pad but the tokenizer does not have a padding token. Please select a token to use as `pad_token` `(tokenizer.pad_token = tokenizer.eos_token e.g.)` or add a new pad token via `tokenizer.add_special_tokens({'pad_token': '[PAD]'})`.

In [ ]:
###Next step: I may have to create a prompt where I would say identify which info is PII and match them to the following columns

In [ ]:
###From what I learned from AI plus training
from datasets import load_dataset

dataset = load_dataset("yelp_review_full")
dataset["train"][100]


from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")


def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True) #create a function to put in the model


tokenized_datasets = dataset.map(tokenize_function, batched=True) #the datset should contain our json or txt file

In [ ]:
small_train_dataset = tokenized_datasets["train"].shuffle(seed=42).select(range(1000))
small_eval_dataset = tokenized_datasets["test"].shuffle(seed=42).select(range(1000)) #selects the dataset and randomly pick the amount

In [ ]:
#Need to add this in to understand the parameters in the model and what to change
from transformers import TrainingArguments

training_args = TrainingArguments(output_dir="test_trainer")

In [9]:
####Comparing it to Microsoft Presidio to identifiy PII info (01/07/2024)
from presidio_analyzer import AnalyzerEngine

# Specify the file path
file_path = "/Users/dkim1998/Downloads/LLM_PII_Project_Code/fake_pii_data_unlabeled_training.txt"

# Read the content of the file
with open(file_path, 'r') as file:
    text_data = file.read()

# Create an instance of the AnalyzerEngine
analyzer = AnalyzerEngine()

# Analyze the text to detect PII
results = analyzer.analyze(text=text_data, language='en')

# Print the results
print("Detected PII:")
for result in results:
    pii_type = result.entity_type
    start_index = result.start
    end_index = result.end
    pii_text = text_data[start_index:end_index]
    print(f"Detected {pii_type}: {pii_text}")

##It works pretty well, much faster but not as perfect

Detected PII:
Detected EMAIL_ADDRESS: fconley@example.net
Detected CREDIT_CARD: 4367458967634
Detected EMAIL_ADDRESS: ylyons@example.com
Detected CREDIT_CARD: 30049209050831
Detected EMAIL_ADDRESS: enelson@example.org
Detected EMAIL_ADDRESS: brian29@example.net
Detected CREDIT_CARD: 6562380614564428
Detected EMAIL_ADDRESS: bellsarah@example.com
Detected CREDIT_CARD: 4807317435372
Detected EMAIL_ADDRESS: wgarcia@example.net
Detected CREDIT_CARD: 5460222294646007
Detected EMAIL_ADDRESS: cdavies@example.com
Detected EMAIL_ADDRESS: kirbysusan@example.org
Detected CREDIT_CARD: 36025271151669
Detected MEDICAL_LICENSE: RJ1802126
Detected EMAIL_ADDRESS: kathleenolson@example.net
Detected CREDIT_CARD: 4918314164928143
Detected EMAIL_ADDRESS: bonnie13@example.org
Detected CREDIT_CARD: 30084465377438
Detected EMAIL_ADDRESS: rbrown@example.net
Detected CREDIT_CARD: 375981331957587
Detected MEDICAL_LICENSE: RZ0515378
Detected EMAIL_ADDRESS: michelle20@example.net
Detected CREDIT_CARD: 6011766720463